<a href="https://colab.research.google.com/github/ChrisT142/FDSFE_CTran/blob/main/Copy_of_FDSfE_kmeans_24Oct23_for_students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

<ipython-input-2-94c4aa801e77>:8: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-whitegrid")


# Get Kaggle dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

!chmod 600 "/content/drive/MyDrive/Kaggle/kaggle (3).json"

import json
envs = json.load(open("/content/drive/MyDrive/Kaggle/kaggle (3).json", "r"))


import os
os.environ["KAGGLE_USERNAME"] = envs['username']
os.environ["KAGGLE_KEY"] = envs['key']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!kaggle datasets list

ref                                                         title                                            size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------  ----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
iamsouravbanerjee/customer-shopping-trends-dataset          Customer Shopping Trends Dataset                146KB  2023-10-05 06:45:37          10675        239  1.0              
zeesolver/consumer-behavior-and-shopping-habits-dataset     Consumer Behavior and Shopping Habits Dataset:  146KB  2023-10-19 13:36:26           1290         29  1.0              
nelgiriyewithana/top-spotify-songs-2023                     Most Streamed Spotify Songs 2023                 47KB  2023-08-26 11:04:57          40501       1160  1.0              
nelgiriyewithana/billionaires-statistics-dataset            Billionaires Statistics Dataset (2023)  

In [7]:
!kaggle datasets download -d camnugent/california-housing-prices -p /content/drive/MyDrive/FDSfE_F2023

california-housing-prices.zip: Skipping, found more recently modified local copy (use --force to force download)


In [8]:
!unzip /content/drive/MyDrive/FDSfE_F2023/california-housing-prices.zip -d /content/drive/MyDrive/FDSfE_F2023

Archive:  /content/drive/MyDrive/FDSfE_F2023/california-housing-prices.zip
replace /content/drive/MyDrive/FDSfE_F2023/housing.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


# Read dataset

In [9]:
df = pd.read_csv("/content/drive/MyDrive/FDSfE_F2023/california-housing-prices.zip")
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


# Select subset of data and scale it

In [10]:
X = df.loc[:, ["median_income", "latitude", "longitude"]]
X

,median_income,latitude,longitude
0,8.3252,37.88,-122.23
1,8.3014,37.86,-122.22
2,7.2574,37.85,-122.24
3,5.6431,37.85,-122.25
4,3.8462,37.85,-122.25
...,...,...,...
20635,1.5603,39.48,-121.09
20636,2.5568,39.49,-121.21
20637,1.7000,39.43,-121.22
20638,1.8672,39.43,-121.32


In [11]:
X.describe()

,median_income,latitude,longitude
count,20640.000000,20640.000000,20640.000000
mean,3.870671,35.631861,-119.569704
std,1.899822,2.135952,2.003532
min,0.499900,32.540000,-124.350000
25%,2.563400,33.930000,-121.800000
50%,3.534800,34.260000,-118.490000
75%,4.743250,37.710000,-118.010000
max,15.000100,41.950000,-114.310000


In [ ]:
scaler = REPLACEME
scaler.fit(REPLACEME)
X[X.columns] = scaler.transform(REPLACEME)
X.describe()

# Cluster the data with kmeans

In [ ]:
kmeans = REPLACEME(n_clusters=6)
X["REPLACEME"] = kmeans.fit_predict(REPLACEME)
X["REPLACEME"] = X["REPLACEME"].astype("category")

X.head()

# Visualize the clusters locations

In [ ]:
sns.relplot(
    x="REPLACEME", y="REPLACEME", hue="REPLACEME", data=REPLACEME, height=6,
)

# Visualize the cluster values

In [ ]:
X["REPLACEME"] = df["REPLACEME"]
sns.catplot(x="REPLACEME", y="REPLACEME", data=REPLACEME, kind="boxen", height=6);